## 기울기소멸 

### 고요속의 외침 

`-` https://www.youtube.com/watch?v=ouitOnaDtFY

`-` 중간에 한명이라도 잘못 말한다면.. 

### 정의 

`-` In machine learning, the vanishing gradient problem is encountered when training artificial neural networks with gradient-based learning methods and backpropagation. 

### 이해

`-` 당연한것 아닌가? 
- 그레디언트 기반의 학습 (그레디언트 기반의 옵티마이저): 손실함수의 기울기를 통하여 업데이트 하는 방식 
- 역전파: 손실함수의 기울기를 구하는 테크닉 (체인룰 + $\alpha$). 구체적으로는 (1) 손실함수를 여러단계로 쪼개고 (2) 각 단계의 미분값을 각각 구하고 (3) 그것들을 모두 곱하여 기울기를 계산한다. 
- 0 근처의 숫자를 계속 곱하면 터지거나 0으로 간다. (사실 안정적인 기울기가 나올 것이라고 생각하는것 자체가 사실 이상함) 

In [1]:
import numpy as np 

In [11]:
grads = np.random.uniform(low=-2,high=2,size=100) 
grads

array([-1.2167476 , -1.12064134,  1.00393117,  1.40056194,  1.92882907,
       -0.09171322,  0.30572762,  1.56992117,  1.32243833, -0.92432003,
       -0.50248891,  1.66943431, -1.60460124,  0.35229854,  1.2274395 ,
        1.37389022,  0.29056595,  0.98707261, -1.2251744 ,  1.80257753,
        0.71537336, -1.15877464, -0.83494789,  1.64240574, -1.32446422,
       -0.89313588, -1.51753367, -0.98460436, -1.56692271, -1.46442158,
        0.68353486,  1.72871458, -0.2368262 , -1.63323852,  0.2627509 ,
        0.5877175 , -0.00262332,  0.11535328, -1.50135131,  1.66543865,
       -1.89538258, -1.28429374,  1.02493065, -0.32715551,  1.30725127,
       -0.33500648, -1.71877426,  1.88252857,  0.06692171, -1.63322342,
       -1.74570482,  1.62121875,  1.97338714, -0.81678251, -0.85935999,
       -0.09854208,  0.49107484, -0.73930724, -0.90959205, -1.94458844,
       -0.29952569,  0.88053565, -0.02598754,  0.58950273, -0.74198508,
        1.79651259, -0.19150611,  0.05414303, -0.69770824,  1.80

In [12]:
grads.prod()

-4.8946716297614096e-17

- 기울기가 소멸함 

In [31]:
grads = np.random.uniform(low=-5,high=5,size=100) 
grads.prod()

-2.0889172052036512e+33

- 기울기가 폭발함. 

In [49]:
grads = np.random.uniform(low=-1,high=3.5,size=100) 
grads.prod()

16364179.47497398

`-` 도깨비: 기울기가 소멸하기도 하고 터지기도 한다. 

### 해결책 (기울기 소멸에 대한 해결책) 

> Note: 잘 정리되어있는것이 없어서 제 머리속에서 정리했습니다. 부정확할수도 있어요 

`-` 개념 
- 데이터 $\to$ (아키텍처,손실함수,역전파,업데이트) 

`-` Multi-level hierarchy
- 여러층을 쪼개서 학습하자 $\to$ 어떻게? 사전학습, 층벼학습 
- 기울기소실문제를 해결하여 딥러닝을 유행시킨 태초의(?) 방법임. 
- 결국 입력자료를 바꾼뒤에 학습하는 형태 

`-` Faster hardware
- GPU를 중심으로 한 테크닉 
- 근본적인 문제해결책은 아니라는 힌튼의 비판 
- CPU를 쓸때보다 GPU를 쓰면 약간 더 깊은 모형을 학습할 수 있다 정도? 

`-` Residual networks
- 훌륭한 접근법중 하나임 
- 아키텍처를 변경하는 방법이지만, 사실상 손실함수를 부드럽게 만드는 기법으로 이해해도 된다. 
- 솟컷이라는 아키텍처를 추가하여 이리저리 실험해보니까 손실함수가 부드러워졌다.  <--- 이런게 아니고 
- 손실함수를 부드럽게 하기 위해서는 층별의차이(residual)를 학습하는게 유리할 것 같다. 그런데 이것을 위한 효과를 주기 위해서는 단지 아키텍처에 숏만만 추가하면 되겠다. <--- 이런 모티브였을 것이다. 

`-` Other activation functions
- 렐루의 개발

`-` 배치정규화 
- 어쩌다보니 되는것. 
- 배치정규화는 원래 공변량 쉬프트를 잡기 위한 방법임. 그런데 기울기 소멸에도 효과가 있음. 현재는 기울기소멸문제에 대한 해결책으로 빠짐없이 언급되고 있음. 2015년의 원래 논문에는 기울기소멸에 대한 언급은 없었음. (https://arxiv.org/pdf/1502.03167.pdf)
- 심지어 배치정규화는 오버피팅을 잡는효과도 있음 (이것은 논문에 언급했음) 

`-` **기울기를 안구하면 안되나?**`
- 베이지안 최적화기법: (https://arxiv.org/pdf/1807.02811.pdf) $\to$ GPU를 어떻게 쓰지? $\to$ 느리다 

## 추천시스템 

### import 

In [56]:
import torch 
import pandas as pd 
from fastai.collab import * 
from fastai.tabular.all import * 

### data (시뮬레이션) 

In [58]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/2021BDA/master/_notebooks/2021-11-30-recommend.csv')
df

,user,item,rating,item_name
0,1,16,1.407223,홍차7
1,1,9,3.949605,커피10
2,1,4,3.925618,커피5
3,1,8,3.889488,커피9
4,1,6,4.215800,커피7
...,...,...,...,...
953,100,11,3.886853,홍차2
954,100,18,3.851641,홍차9
955,100,17,3.588314,홍차8
956,100,16,3.927570,홍차7
